In [3]:
import os

os.makedirs(os.path.join('data'), exist_ok=True)
data_file = os.path.join('data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms, Alley, Price\n')
    f.write('NA, Pave, 127500\n')
    f.write('2, NA, 106000\n')
    f.write('4, NA, 178100\n')
    f.write('NA, NA, 140000\n')

常用pandas来读取数据

In [4]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms  Alley   Price
0       NaN   Pave  127500
1       2.0     NA  106000
2       4.0     NA  178100
3       NaN     NA  140000


为了处理缺失的数据，典型方法是删除（直接删除有缺失数据的一行元素）和插值。这里考虑插值。

把0、1列作为inputs，2列作为outputs

1. 对于**连续值**，把 inputs 列中 NA 的值用列的平均值填充

In [10]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms  Alley
0       3.0   Pave
1       2.0     NA
2       4.0     NA
3       3.0     NA


C:\Users\acbgzm\anaconda3\envs\zm_torch\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


2. 对于inputs中的**类别值或离散值**，将NAN视为一个类别来对待

In [11]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms   Alley_ NA   Alley_ Pave   Alley_nan
0       3.0           0             1           0
1       2.0           1             0           0
2       4.0           1             0           0
3       3.0           1             0           0


现在 inputs 和 outputs 中的所有条目都是数值类型，可以转换成张量了。

In [12]:
import torch

X, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
X, y

(tensor([[3., 0., 1., 0.],
         [2., 1., 0., 0.],
         [4., 1., 0., 0.],
         [3., 1., 0., 0.]], dtype=torch.float64),
 tensor([127500, 106000, 178100, 140000]))

#### QA中的注意要点

1. reshape相当于创建一个view，而不是创建一个新地址的对象

In [16]:
a = torch.arange(12)
b = a.reshape((3, 4))
b[:] = 2
a, b

(tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
 tensor([[2, 2, 2, 2],
         [2, 2, 2, 2],
         [2, 2, 2, 2]]))